In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy.spatial
import pandas as pd
import sklearn.decomposition
import matplotlib.pyplot as plt
# import keras
from sklearn import preprocessing
from sklearn.metrics import pairwise_distances,mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from utils.readProfiles import readMergedProfiles
from utils.pred_models import *
from utils.saveAsNewSheetToExistingFile import saveAsNewSheetToExistingFile
from multiprocessing import Pool

sns.set_style("whitegrid")
# from utils import networksEvol, tsne, readProfiles
import umap

# Load data



- CDRP-BBBC047-Bray-CP-GE (Cell line: U2OS):
    * There are 30,430 and 21,782 unique compounds for CP and GE datasets, respectively.
    * Median number of replicates for each dataset is as follows: CP: ~4 , GE: ~3. 
    * 20,358 compounds are present in both datasets.
    * Replicate Level Shapes (nSamples x nFeatures): cp:  (153386 , 1783) ,  ge:  (68120 , 977)
    * Treatment Level Shapes (nSamples x nFeatures): cp:  (30430, 1786)   ,  ge:  (21782, 981) 
    * Merged Profiles Shape:                              (20358, 2766)
    * High Rep corr: CP: 30618 to 7892, l1k: 21069 to 2870, overlap:  3
    
    
- CDRPBIO-BBBC036-Bray-CP-GE (Cell line: U2OS):
    * There are 30,430 and 21,782 unique compounds for CP and GE datasets, respectively.
    * Median number of replicates for each dataset is as follows: CP: ~4 , GE: ~3. 
    * 20,358 compounds are present in both datasets.
    * Replicate Level Shapes (nSamples x nFeatures): cp:  (153386 , 1783) ,  ge:  (68120 , 977)
    * Treatment Level Shapes (nSamples x nFeatures): cp:  (30430, 1786)   ,  ge:  (21782, 981) 
    * Merged Profiles Shape:                   
    * High Rep corr: CP: 2239 to 312, l1k: 1535 to 448, overlap:  131


- LUAD-BBBC041-Caicedo-CP-GE (Cell line: A549) : 
    * There are 593 and 529 unique alleles for CP and GE datasets, respectively.
    * Median number of replicates for each dataset is as follows: CP: ~8, GE: ~8.
    * 525 alleles are present in both datasets.
    * Replicate Level Shapes (nSamples x nFeatures): cp:  (6144 , 1783) ,  ge:  (4232 , 978)
    * Treatment Level Shapes (nSamples x nFeatures): cp: (593, 1786) , ge: (529, 979) 
    * Merged Profiles Shape:                             (525, 2764)    
    * High Rep corr: CP: x to x, l1k: x to x, overlap:  x    
    
    
- TA-ORF-BBBC037-Rohban-CP-GE (Cell line: U2OS) :
    * There are 299 and 226 number of unique compounds for CP and GE datasets respectively.
    * Median number of replicates for each dataset is as follows: CP: ~5 , GE: ~2.
    * 188 alleles are present in both datasets.
    * Replicate Level Shapes (nSamples x nFeatures):         cp: (1920 , 1783) ,  ge: (729 , 978)
    * Treatment Level Shapes (nSamples x nFeatures+metadata):cp: (323, 1784)   ,  ge: (328, 979)
    * Merged Profiles Shape:                                     (149, 2762)
    * High Rep corr: CP: x to x, l1k: x to x, overlap:  x    
    
    
- LINCS-Pilot1-CP-GE (Cell line: A549) :
    * There are 1570 unique compounds across 7 doses for CP dataset
    * There are x unique compounds across 7 doses for GE dataset
    * Median number of replicates for each dataset is as follows: CP: ~5 , GE: ~3.
    * 6984 "compounds-dose" are present in both datasets. 
    * Replicate Level Shapes (nSamples x nFeatures):         cp: (52223 , 1747) ,  ge: (27837 , 978)
    * Treatment Level Shapes (nSamples x nFeatures+metadata):cp: (9394, 1748)   ,  ge: (8370, 979)
    * Merged Profiles Shape:                                     (6984, 2726)
    * High Rep corr: CP: 9394 to 4647, l1k: 8369  to  2338, overlap:  1140
    

#### Paths

In [ ]:
procProf_dir='/home/ubuntu/datasetsbucket/Rosetta-GE-CP/'
# procProf_dir='/home/ubuntu/bucket/projects/2018_04_20_Rosetta/workspace/'
metadata_dir='/home/ubuntu/bucket/projects/2018_04_20_Rosetta/workspace/metadata/'
results_dir='./results/'


## Prediction of single GE expression levels based on the full CP profiles

In [ ]:
################################################
# dataset options: 'CDRP' , 'LUAD', 'TAORF', 'LINCS', 'CDRP-bio'
datasets=['TAORF'];
# datasets=['LINCS', 'CDRP-bio','CDRP'];

DT_kfold={'LUAD':10, 'TAORF':5, 'LINCS':25, 'CDRP-bio':6,'CDRP':40}

################################################
# CP Profile Type options: 'augmented' , 'normalized', 'normalized_variable_selected'
profileType='normalized_variable_selected'

################################################
# filtering to compounds which have high replicates for both GE and CP datasets
# highRepOverlapEnabled=0
# 'highRepUnion','highRepOverlap'
filter_perts='highRepOverlap'

################################################
pertColName='PERT'
profileLevel='treatment'; #'replicate'  or  'treatment'
if highRepOverlapEnabled:
    f='filt'
else:
    f=''
    
# def f(dataset):
for dataset in datasets:
   
    
    nRep=2
    mergProf_repLevel,mergProf_treatLevel,cp_features,l1k_features=\
    readMergedProfiles(procProf_dir,dataset,profileType,profileLevel,nRep,highRepOverlapEnabled);


#     if profileLevel=='replicate':
#         l1k=mergProf_repLevel[[pertColName]+l1k_features]
#         cp=mergProf_repLevel[[pertColName]+cp_features]
#     elif profileLevel=='treatment':
    l1k=mergProf_treatLevel[[pertColName]+l1k_features]
    cp=mergProf_treatLevel[[pertColName]+cp_features]


    if dataset=='LINCS':     
        cp['Compounds']=cp['PERT'].str[0:13]
        l1k['Compounds']=l1k['PERT'].str[0:13]
    else:
        cp['Compounds']=cp['PERT']
        l1k['Compounds']=l1k['PERT']      


    le = preprocessing.LabelEncoder()
    group_labels=le.fit_transform(l1k['Compounds'].values)


    scaler_ge = preprocessing.StandardScaler()
    scaler_cp = preprocessing.StandardScaler()
    l1k_scaled=l1k.copy()
    l1k_scaled[l1k_features] = scaler_ge.fit_transform(l1k[l1k_features].values)
    cp_scaled=cp.copy()
    cp_scaled[cp_features] = scaler_cp.fit_transform(cp[cp_features].values.astype('float64'))
    
    
    for model in ["MLP"]:#["Lasso","MLP","RFR"]:    

        if 1:
            cp=cp_scaled.copy()
            l1k=l1k_scaled.copy()

        ##############################
         
        k_fold=DT_kfold[dataset]
            
        pred_df=pd.DataFrame(index=range(k_fold),columns=l1k_features)
        pred_df_rand=pd.DataFrame(index=range(k_fold),columns=l1k_features)
        ii=0
        for l in l1k_features:
            ii+=1
            print(ii)
            if model=="Lasso":
                scores,scores_rand=lasso_cv_plus_model_selection(cp[cp_features],l1k[l],k_fold,group_labels)        
#                 scores,scores_rand=lasso_cv(cp[cp_features],l1k[l],k_fold,group_labels)
            elif model=="MLP":
                scores,scores_rand=MLP_cv_plus_model_selection(cp[cp_features],l1k[l],k_fold,group_labels)
#                 scores,scores_rand=MLP_cv(cp[cp_features],l1k[l],k_fold,group_labels)  
            elif model=="RFR":
                scores,scores_rand=RFR_cv_plus_model_selection(cp[cp_features],l1k[l],k_fold,group_labels)  
            
            pred_df[l]=scores
            pred_df_rand[l]=scores_rand

#         gafgfdssgfd
        ########################### mapping prob_ids to genes names    
        meta=pd.read_csv("/home/ubuntu/bucket/projects/2018_04_20_Rosetta/workspace/metadata/affy_probe_gene_mapping.txt",delimiter="\t",header=None, names=["probe_id", "gene"])
        meta_gene_probID=meta.set_index('probe_id')
        d = dict(zip(meta_gene_probID.index, meta_gene_probID['gene']))
        pred_df = pred_df.rename(columns=d)    
        pred_df_rand = pred_df_rand.rename(columns=d)  


        meltedPredDF=pd.melt(pred_df).rename(columns={'variable':'lmGens','value':'pred score'})
        meltedPredDF_rand=pd.melt(pred_df_rand).rename(columns={'variable':'lmGens','value':'pred score'})
        meltedPredDF['d']="n-folds"
        meltedPredDF_rand['d']="random"
        filename=results_dir+'/SingleGenePred/scores.xlsx'
        
        profTypeAbbrev=''.join([s[0] for s in profileType.split('_')])
        
        saveAsNewSheetToExistingFile(filename,pd.concat([meltedPredDF,meltedPredDF_rand],ignore_index=True),\
                                     model+'-'+dataset+'-'+profTypeAbbrev+'-'+f+'-'+str(k_fold)+'-ht')
        
#     return 

# with Pool(10) as p:
#     p.map(f, datasets)

## Prediction of single CP features based on the full GE profiles

In [ ]:
################################################
# dataset options: 'CDRP' , 'LUAD', 'TAORF', 'LINCS', 'CDRP-bio'
# datasets=['LUAD', 'TAORF', 'LINCS', 'CDRP-bio'];
datasets=['TAORF','CDRP'];

# DT_kfold={'LUAD':10, 'TAORF':5, 'LINCS':20, 'CDRP-bio':20,'CDRP':40}

DT_kfold={'LUAD':9, 'TAORF':5, 'LINCS':25, 'CDRP-bio':6,'CDRP':40}


################################################
# CP Profile Type options: 'augmented' , 'normalized', 'normalized_variable_selected'
# 'normalized_feature_select_dmso'
profileType='normalized'

################################################
# filtering to compounds which have high replicates for both GE and CP datasets
# highRepOverlapEnabled=0
# 'highRepUnion','highRepOverlap'
filter_perts='highRepOverlap'


################################################
pertColName='PERT'
profileLevel='treatment'; #'replicate'  or  'treatment'
if filter_perts:
    f='filt'
else:
    f=''
    
# def f(dataset):
for dataset in datasets:
        
    
    nRep=2
    mergProf_repLevel,mergProf_treatLevel,cp_features,l1k_features=\
    readMergedProfiles(procProf_dir,dataset,profileType,profileLevel,nRep,filter_perts);


    l1k=mergProf_treatLevel[[pertColName]+l1k_features]
    cp=mergProf_treatLevel[[pertColName]+cp_features]

        
    if dataset=='LINCS':     
        cp['Compounds']=cp['PERT'].str[0:13]
        l1k['Compounds']=l1k['PERT'].str[0:13]
    else:
        cp['Compounds']=cp['PERT']
        l1k['Compounds']=l1k['PERT']      


    le = preprocessing.LabelEncoder()
    group_labels=le.fit_transform(l1k['Compounds'].values)        
        

    scaler_ge = preprocessing.StandardScaler()
    scaler_cp = preprocessing.StandardScaler()
    l1k_scaled=l1k.copy()
    l1k_scaled[l1k_features] = scaler_ge.fit_transform(l1k[l1k_features].values)
    cp_scaled=cp.copy()
    cp_scaled[cp_features] = scaler_cp.fit_transform(cp[cp_features].values.astype('float64'))

    for model in ["Lasso", "MLP"]:    
    
#         if model=="MLP":
#             cp_scaled[cp_features] =preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit_transform(cp_scaled[cp_features].values)   
#             l1k_scaled[l1k_features] =preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit_transform(l1k_scaled[l1k_features].values)           

        if 1:
            cp=cp_scaled.copy()
            l1k=l1k_scaled.copy()

        ##############################
            
            
        k_fold=DT_kfold[dataset]
#         k_fold=int(np.unique(group_labels).shape[0]/20)            
        pred_df=pd.DataFrame(index=range(k_fold),columns=cp_features)
        pred_df_rand=pd.DataFrame(index=range(k_fold),columns=cp_features)
        ii=0
        for c in cp_features:
            ii+=1
            print(ii)            
            if model=="Lasso":
                scores,scores_rand=lasso_cv_plus_model_selection(l1k[l1k_features],cp[c],k_fold,group_labels)
            elif model=="MLP":
                scores,scores_rand=MLP_cv_plus_model_selection(l1k[l1k_features],cp[c],k_fold,group_labels)
            pred_df[c]=scores
            pred_df_rand[c]=scores_rand

            
        meltedPredDF=pd.melt(pred_df).rename(columns={'variable':'CP-Features','value':'pred score'})
        meltedPredDF_rand=pd.melt(pred_df_rand).rename(columns={'variable':'CP-Features','value':'pred score'})
        meltedPredDF['d']="n-folds"
        meltedPredDF_rand['d']="random"
#         filename='../../results/SingleCPfeatPred/scores.xlsx'
        filename=results_dir+'/SingleCPfeatPred/scores.xlsx'
        
        profTypeAbbrev=''.join([s[0] for s in profileType.split('_')])        
        
        saveAsNewSheetToExistingFile(filename,pd.concat([meltedPredDF,meltedPredDF_rand],ignore_index=True),\
                                     model+'-'+dataset+'-'+profTypeAbbrev+'-'+f+'-'+str(k_fold)+'-ht')


    
#     return 

# with Pool(10) as p:
#     p.map(f, datasets)        

### CP Category specific scores for single gene prediction - based on Lasso  regression

In [ ]:
%%capture
################################################
# dataset options: 'CDRP' , 'LUAD', 'TAORF', 'LINCS', 'CDRP-bio'
# datasets=['LUAD', 'TAORF', 'LINCS', 'CDRP-bio'];
datasets=['LINCS'];

# DT_kfold={'LUAD':10, 'TAORF':5, 'LINCS':20, 'CDRP-bio':20,'CDRP':40}

DT_kfold={'LUAD':9, 'TAORF':5, 'LINCS':25, 'CDRP-bio':6,'CDRP':40}


################################################
# CP Profile Type options: 'augmented' , 'normalized', 'normalized_variable_selected'
# 'normalized_feature_select_dmso'
profileType='normalized'

################################################
# filtering to compounds which have high replicates for both GE and CP datasets
# highRepOverlapEnabled=0
# 'highRepUnion','highRepOverlap'
filter_perts='highRepOverlap'


################################################
pertColName='PERT'
profileLevel='treatment'; #'replicate'  or  'treatment'
if filter_perts:
    f='filt'
else:
    f=''
    
# def f(dataset):
for dataset in datasets:
        
    
    nRep=2
    mergProf_repLevel,mergProf_treatLevel,cp_features,l1k_features=\
    readMergedProfiles(procProf_dir,dataset,profileType,profileLevel,nRep,filter_perts);


    l1k=mergProf_treatLevel[[pertColName]+l1k_features]
    cp=mergProf_treatLevel[[pertColName]+cp_features]

        
    if dataset=='LINCS':     
        cp['Compounds']=cp['PERT'].str[0:13]
        l1k['Compounds']=l1k['PERT'].str[0:13]
    else:
        cp['Compounds']=cp['PERT']
        l1k['Compounds']=l1k['PERT']      


    le = preprocessing.LabelEncoder()
    group_labels=le.fit_transform(l1k['Compounds'].values)        
        

    scaler_ge = preprocessing.StandardScaler()
    scaler_cp = preprocessing.StandardScaler()
    l1k_scaled=l1k.copy()
    l1k_scaled[l1k_features] = scaler_ge.fit_transform(l1k[l1k_features].values)
    cp_scaled=cp.copy()
    cp_scaled[cp_features] = scaler_cp.fit_transform(cp[cp_features].values.astype('float64'))

    if 1:
        cp=cp_scaled.copy()
        l1k=l1k_scaled.copy()

    ##############################

    # k_fold=10
    k_fold=DT_kfold[dataset]
    Channelss=['DNA','RNA','AGP','Mito','ER']
    featureGroups=['Texture','Intensity','RadialDistribution']
    relationMat_mpCat=pd.DataFrame(index=l1k_features)
    for ch in range(len(Channelss)):
        for f in range(len(featureGroups)):    
            selectedCols=cp.columns[cp.columns.str.contains(Channelss[ch]) &\
                           cp.columns.str.contains(featureGroups[f]) &\
                           cp.columns.str.contains("Cells_|Cytoplasm_|Nuclei_")].tolist();
            if selectedCols!=[]:
                for l in l1k_features:
#                     scores,scores_rand=lasso_cv_plus_model_selection(cp[selectedCols],l1k[l],k_fold,group_labels,0)
                    scores,scores_rand=MLP_cv_plus_model_selection(cp[selectedCols],l1k[l],k_fold,group_labels,0)

                    relationMat_mpCat.loc[l,Channelss[ch]+'_'+featureGroups[f]]=np.median(scores)

    Channelss=['Nuclei','Cytoplasm','Cells']
    featureGroups=['AreaShape']

    for ch in range(len(Channelss)):
        for f in range(len(featureGroups)):    
            selectedCols=cp.columns[cp.columns.str.contains(Channelss[ch]) &\
                           cp.columns.str.contains(featureGroups[f]) &\
                           cp.columns.str.contains("Cells_|Cytoplasm_|Nuclei_")].tolist();
            if selectedCols!=[]:
                for l in l1k_features:
#                     scores,scores_rand=lasso_cv_plus_model_selection(cp[selectedCols],l1k[l],k_fold,group_labels,0)
                    scores,scores_rand=MLP_cv_plus_model_selection(cp[selectedCols],l1k[l],k_fold,group_labels,0)
                    
                    relationMat_mpCat.loc[l,Channelss[ch]+'_'+featureGroups[f]]=np.median(scores)

    ########################### mapping prob_ids to genes names    
    meta=pd.read_csv("/home/ubuntu/bucket/projects/2018_04_20_Rosetta/workspace/metadata/affy_probe_gene_mapping.txt",delimiter="\t",header=None, names=["probe_id", "gene"])
    meta_gene_probID=meta.set_index('probe_id')
    d = dict(zip(meta_gene_probID.index, meta_gene_probID['gene']))
    relationMat_mpCat = relationMat_mpCat.rename(index=d)    


    filename=results_dir+'/SingleGenePred_cpCategoryMap/cat_scores_maps.xlsx'

    profTypeAbbrev=''.join([s[0] for s in profileType.split('_')])        

    saveAsNewSheetToExistingFile(filename,relationMat_mpCat,dataset+'-'+str(k_fold)+'-MLP-ht')    
    

/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/sit

/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/sit

/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/sit

/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/sit

/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/sit

...

/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/sit

/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/sit

/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/sit

/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/sit

/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.8/sit

In [ ]:
cp.columns[cp.columns.str.contains('RadialDistribution')]

In [ ]:

Channelss[ch],featureGroups[f]

In [ ]:
selectedCols!=[]

In [ ]:
# plt.figure()
# f, axes = plt.subplots(figsize=(15,4))
# ax = sns.heatmap(topGenes_df)
clustergrid=sns.clustermap(topGenes_df.T,xticklabels=1,yticklabels=1,figsize=(50, 5),cmap="coolwarm",cbar_pos=(1, 0.2, .03, .4))


In [ ]:
# clustergrid=sns.clustermap(topGenes_df2.T,xticklabels=1,yticklabels=1,figsize=(10, 5),cmap="vlag",\
#                            vmin=-1, vmax=1,cbar_pos=(1, 0.2, .03, .4))
%matplotlib inline
clustergrid=sns.clustermap(topGenes_df2.T,xticklabels=1,yticklabels=1,figsize=(13, 5),cmap="vlag",\
                           vmin=-1, vmax=1,cbar_pos=(1, 0.2, .03, .4))
clustergrid.savefig('../../results/cpCategoriesHM-'+dataset+'.png', format='png',dpi=100)

In [ ]:
%matplotlib inline
clustergrid=sns.clustermap(topGenes_df2.T,xticklabels=1,yticklabels=1,figsize=(20, 5),cmap="vlag",\
                           vmin=0, vmax=0.8,cbar_pos=(1, 0.2, .03, .4))
# clustergrid.savefig('../../results/cpCategoriesHM-'+dataset+'.png', format='png',dpi=100)

In [ ]:
topGenes_mito_radial=relationMat_mpCat[relationMat_mpCat["Mito_RadialDistribution"]>0.6]#[["Mito_RadialDistribution"]]
clustergrid=sns.clustermap(topGenes_mito_radial.T,xticklabels=1,yticklabels=1,figsize=(20, 5),cmap="vlag",\
                           vmin=-0.8, vmax=0.8,cbar_pos=(1, 0.2, .03, .4))

In [ ]:
topGenes_mito_radial.sort_values(by='Mito_RadialDistribution').index.values